In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
import os
from dotenv import load_dotenv
import csv
import time
from selenium.webdriver.support.ui import Select
from datetime import date, timedelta
from threading import Thread
import threading
from typing import List

In [4]:
load_dotenv()
edge_driver_path = os.getenv('EDGE_DRIVER_PATH')
driver = webdriver.Edge(service=Service(edge_driver_path))
driver2 = webdriver.Edge(service=Service(edge_driver_path))
driver3 = webdriver.Edge(service=Service(edge_driver_path))
lock = threading.Lock()
list_kpop_mv_url = "https://kpop.daisuki.com.br/en/mvs.html?options=101&tags=dhcmoravxpnezjsl&cols=101110000&ord=5D&fbclid=IwAR2rpByeOSp6GktqkzWLI5ipwDLLit6IE3qEhdpLemqmeBtn8C6xiLRQYo8"
detail_url = "https://kpop.daisuki.com.br/mv.php?layout=1&id="

# prepare vpn for each driver

In [4]:
driver.get(list_kpop_mv_url)

In [5]:
more_button = driver.find_element(By.XPATH, '//*[@id="morebtn"]')
driver.execute_script("arguments[0].click();", more_button)

In [6]:
list_size_cbb = driver.find_element(By.XPATH, '//*[@id="hlimit"]')
select = Select(list_size_cbb)
select.select_by_visible_text('500 items')

published_date_inp_start = driver.find_element(By.XPATH, '//*[@id="fgyear"]')
published_date_inp_end = driver.find_element(By.XPATH, '//*[@id="fgyeared"]')

start_date = date(2014, 1, 1)
end_date = date(2024, 1, 1)
list_popup_id = []
while start_date <= end_date:
    published_date_inp_start.clear()
    published_date_inp_start.send_keys(start_date.strftime('%Y-%m-%d'))

    iteration_end_date = start_date + timedelta(days=30)
    if iteration_end_date > end_date:
        iteration_end_date = end_date

    published_date_inp_end.clear()
    published_date_inp_end.send_keys(iteration_end_date.strftime('%Y-%m-%d'))

    print(f"Start date: {start_date.strftime('%Y-%m-%d')}")
    print(f"End date: {iteration_end_date.strftime('%Y-%m-%d')}")

    update_list_btn = driver.find_element(By.XPATH, '//*[@id="btngenerate"]')
    driver.execute_script("arguments[0].click();", update_list_btn)

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'list_tiles_item')))
    list_items = driver.find_elements(By.CLASS_NAME, 'list_tiles_item')
    print(f"Number of items: {len(list_items)}")


    for item in list_items:
        id = item.get_attribute('id')
        list_popup_id.append(int(id.replace('v', '')))

    start_date += timedelta(days=30)
filename = 'popup_id_data.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for id in list_popup_id:
        writer.writerow([id])


Start date: 2014-01-01
End date: 2014-01-31
Number of items: 65
Start date: 2014-01-31
End date: 2014-03-02
Number of items: 71
Start date: 2014-03-02
End date: 2014-04-01
Number of items: 68
Start date: 2014-04-01
End date: 2014-05-01
Number of items: 43
Start date: 2014-05-01
End date: 2014-05-31
Number of items: 90
Start date: 2014-05-31
End date: 2014-06-30
Number of items: 85
Start date: 2014-06-30
End date: 2014-07-30
Number of items: 79
Start date: 2014-07-30
End date: 2014-08-29
Number of items: 104
Start date: 2014-08-29
End date: 2014-09-28
Number of items: 71
Start date: 2014-09-28
End date: 2014-10-28
Number of items: 73
Start date: 2014-10-28
End date: 2014-11-27
Number of items: 100
Start date: 2014-11-27
End date: 2014-12-27
Number of items: 62
Start date: 2014-12-27
End date: 2015-01-26
Number of items: 77
Start date: 2015-01-26
End date: 2015-02-25
Number of items: 60
Start date: 2015-02-25
End date: 2015-03-27
Number of items: 96
Start date: 2015-03-27
End date: 2015-

In [9]:
list_popup_id = []

filename = 'popup_id_data.csv'
# Read from the file
with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        list_popup_id.append(row[0])

In [10]:
class VideoData:
    def __init__(self, title, release_date, youtube_id, artist_name, organic_views, likes, artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers):
        self.title = title
        self.release_date = release_date
        self.youtube_id = youtube_id
        self.artist_name = artist_name
        self.organic_views = organic_views
        self.likes = likes
        self.artist_organic_views = artist_organic_views
        self.artist_debut_date = artist_debut_date
        self.branch_physical_sales = branch_physical_sales
        self.youtube_followers = youtube_followers

    def __str__(self):
        return f"title: {self.title}, release_date: {self.release_date}, youtube_id: {self.youtube_id}, artist_name: {self.artist_name}, organic_views: {self.organic_views}, likes: {self.likes}, artist_organic_views: {self.artist_organic_views}, artist_debut_date: {self.artist_debut_date}, branch_physical_sales: {self.branch_physical_sales}, youtube_followers: {self.youtube_followers}"

class ArtistData:
    def __init__(self, artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers):
        self.artist_organic_views = artist_organic_views
        self.artist_debut_date = artist_debut_date
        self.branch_physical_sales = branch_physical_sales
        self.youtube_followers = youtube_followers

In [11]:
drivers = [driver, driver2, driver3]
start_idx = 0
end_idx = 2000
length = len(list_popup_id)
list_popup_id = list_popup_id[start_idx:end_idx]
part1 = list_popup_id[0:len(list_popup_id)//3]
part2 = list_popup_id[len(list_popup_id)//3:2*len(list_popup_id)//3]
part3 = list_popup_id[2*len(list_popup_id)//3:]

searched_artist_data = {}
process = 0


In [12]:
def process_crawl_details(driver: WebDriver, ids):
    global process
    videoData_list: List[VideoData] = []
    idx = 0
    for id in ids:
        driver.get(detail_url + str(id))
        driver.execute_script("expandcontent('expcontent','morebtn');")

        type = driver.find_element(By.XPATH, '//li[contains(text(), "Type: ")]/b').text
        if type != 'Main':
            continue

        artist_name = driver.find_element(By.XPATH, '//li[contains(text(), "Artist: ")]/b/a').text
        title = driver.find_element(By.TAG_NAME, 'h1').text.replace('Video Details - ', '').replace(f' - {artist_name}', '')
        release_date = driver.find_element(By.XPATH, '//li[contains(text(), "Youtube Channel: ")]/b').text
        artist_url = driver.find_element(By.XPATH, '//li[contains(text(), "Artist: ")]/b/a').get_attribute('href')
        youtube_id = driver.find_element(By.XPATH, '//li[contains(text(), "Youtube ID:	")]/b/a').text
        
        view_like_row = driver.find_elements(By.CLASS_NAME, 'dashbottom')
        organic_views = 0
        likes = 0
        try:
            driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/h3')
            for i in range(len(view_like_row) - 29, len(view_like_row)):
                organic_views = driver.find_element(By.XPATH, f'/html/body/div[2]/div[2]/table/tbody/tr[{i}]/td[2]').text
                likes = driver.find_element(By.XPATH, f'/html/body/div[2]/div[2]/table/tbody/tr[{i}]/td[3]').text
                if organic_views == '-' or likes == '-':
                    continue
                organic_views += int(organic_views.replace('.', ''))
                likes += int(likes.replace('.', ''))
        except:
            pass
        time.sleep(5)
        if artist_url in searched_artist_data:
            artist_data = searched_artist_data[artist_url]
        else:
            driver.get(artist_url)
            try:
                artist_organic_views = driver.find_element(By.XPATH, '//*[@id="maincontentwindow"]/article/div[1]/div[2]/table/tbody/tr[2]/td[1]/b').text
            except:
                artist_organic_views = 0
            try:
                artist_debut_date = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Debut Day")]]/td/b').text
            except:
                artist_debut_date = ''
            try:
                branch_physical_sales = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Brand Views")]]/td/b').text
            except:
                branch_physical_sales = ''
            try:
                youtube_followers = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Youtube Followers")]]/td/b').text
            except:
                youtube_followers = ''
            artist_data = ArtistData(artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers)

            with lock:
                searched_artist_data[artist_url] = artist_data

        videoData_list.append(VideoData(title, release_date, youtube_id, artist_name, organic_views, likes, artist_data.artist_organic_views, artist_data.artist_debut_date, artist_data.branch_physical_sales, artist_data.youtube_followers))
        
        with lock:
            if idx % 10 == 0:
                with open('video_data.csv', 'a', newline='', encoding='utf-8') as csvfile:
                    fieldnames = ['title', 'release_date', 'youtube_id', 'artist_name', 'artist_url', 'organic_views', 'likes', 'artist_organic_views', 'artist_debut_date', 'branch_physical_sales', 'youtube_followers']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                    if not os.path.exists('video_data.csv'):
                        writer.writeheader()
                    for video in videoData_list:
                        try:
                            writer.writerow({
                                'title': video.title, 
                                'release_date': video.release_date, 
                                'youtube_id': video.youtube_id, 
                                'artist_name': video.artist_name, 
                                'organic_views': video.organic_views, 
                                'likes': video.likes, 
                                'artist_organic_views': video.artist_organic_views, 
                                'artist_debut_date': video.artist_debut_date, 
                                'branch_physical_sales': video.branch_physical_sales, 
                                'youtube_followers': video.youtube_followers
                            })
                        except Exception as e:
                            print(f"Error writing video data to CSV: {e}")
            videoData_list.clear()
            process += 1

        idx += 1
        
        print(f"thread {threading.get_ident()}, idx: {idx}, title: {title}, release_date: {release_date}, youtube_id: {youtube_id}, artist_name: {artist_name}, artist_url: {artist_url}, organic_views: {organic_views}, likes: {likes}, artist_organic_views: {artist_organic_views}, artist_debut_date: {artist_debut_date}, branch_physical_sales: {branch_physical_sales}, youtube_followers: {youtube_followers}")
        time.sleep(5)

In [13]:
def print_process():
    global length
    global process
    while True:
        print(process)
        time.sleep(10)
print_thread = Thread(target=print_process)
print_thread.start()

thread1 = Thread(target=process_crawl_details, args=(drivers[0], part1))
thread2 = Thread(target=process_crawl_details, args=(drivers[1], part2))
thread3 = Thread(target=process_crawl_details, args=(drivers[2], part3))

thread1.start()
thread2.start()
thread3.start()

# Wait for all threads to finish
thread1.join()
thread2.join()
thread3.join()
print_thread.join()

0
0
0
0
thread 26488, idx: 1, title: mei guan xi, release_date: 1theK (원더케이), youtube_id: 7v8oW6u4t3Y, artist_name: Gary, artist_url: https://kpop.daisuki.com.br/artists/Gary.html, organic_views: 0, likes: 0, artist_organic_views: 67M+, artist_debut_date: 2014-01-15, branch_physical_sales: , youtube_followers: 
1
thread 4120, idx: 1, title: Miniskirt, release_date: AOA (에이오에이), youtube_id: q6f-LLM1H6U, artist_name: AOA, artist_url: https://kpop.daisuki.com.br/artists/AoA.html, organic_views: 0, likes: 0, artist_organic_views: 915.5M+, artist_debut_date: 2012-07-30, branch_physical_sales: 964.398.051+, youtube_followers: 1.310.000+
thread 26488, idx: 2, title: Challenge, release_date: Stone Music Entertainment, youtube_id: iiiWuOCzoao, artist_name: Guckkasten, artist_url: https://kpop.daisuki.com.br/artists/Guckkasten.html, organic_views: 0, likes: 0, artist_organic_views: 5.8M+, artist_debut_date: 2008-12-29, branch_physical_sales: 65.571.278+, youtube_followers: 38.700+
3
3
thread 264